# Advent of Code 2019
## [Day 15: Oxygen System](https://adventofcode.com/2019/day/15)

In [1]:
import aocd
program = [*map(int, (n for n in aocd.get_data(year=2019, day=15).split(',')))]
program[:10]

[3, 1033, 1008, 1033, 1, 1032, 1005, 1032, 31, 1008]

In [2]:
from pathlib import Path
import sys
if str(Path('../..').resolve()) not in sys.path:
    sys.path.insert(0, str(Path('../..').resolve()))

from intcode import Intcode

In [3]:
from enum import IntEnum

In [4]:
import numpy as np
import functools

In [5]:
from collections import deque, namedtuple

### Part 1

In [6]:
translations = {
    'N': np.array([-1, 0]),
    'S': np.array([ 1, 0]),
    'E': np.array([ 0, 1]),
    'W': np.array([ 0,-1]),
}

class Move(IntEnum):
    N = 1
    S = 2
    W = 3
    E = 4
    
class Sense(IntEnum):
    Wall = 0
    Empty = 1
    Goal = 2
    
def get_coords(path, origin=(0,0)):
    steps = map(lambda d: translations[d.name], path)
    return tuple(functools.reduce(lambda x, y: x+y, steps, np.array(origin)))

get_coords([Move.N, Move.E, Move.S])

(0, 1)

In [7]:
for d in Move:
    print(d)

Move.N
Move.S
Move.W
Move.E


In [28]:
def shortest_path_to_oxygen():
    seen = set()
    to_try = deque([(Intcode(program), [])])
    while to_try:
        cpu, path = to_try.popleft()
        if not path:
            result = Sense.Empty
        else:
            coords = get_coords(path)
            if coords in seen:
                continue
            seen.add(coords)
            # print(f"checking {coords}")
            move = path[-1]
            cpu.input.put(move)
            cpu.run(pause_on_output=True)
            result = Sense(cpu.output.get())
        if result == Sense.Goal:
            return path, cpu
        elif result == Sense.Wall:
            continue
        else:
            for d in Move:
                to_try.append((cpu.fork(), path + [d]))
    return "none"

shortest_path, repair_droid = shortest_path_to_oxygen()
''.join([step.name for step in shortest_path])

'NNWWNNWWWWWWSSWWSSWWSSSSSSSSEESSSSSSSSEESSEENNNNEEEEEEEEEENNWWWWNNWWSSWWNNWWSSWWNNNNEENNNNEESSSSEENNNNEENNEEEEEEEENNNNWWSSWWWWNNEENNEEEENNWWNNNNNNWWNNNNWWWWWWNNEEEENNEESSEESSEEEESSSSWWSSEESSSSEEEENNWWNNNNEEEESSEESSSSSSWWSSEESSWWSSEESSWWSSEESSSSSSSSWWNNWWNNEENNWWWWWWNNEENNEENNNNWWSSWWSSWWSSSSSSSSSSSSSSEENNNNNNNNEESSSSSSSSEEEEEENNWWWWNN'

In [21]:
get_coords(shortest_path)

(14, 14)

#### Part 1 Answer
**What is the fewest number of movement commands** required to move the repair droid from its starting position to the location of the oxygen system?

In [22]:
len(shortest_path)

336

### Part 2

In [32]:
def longest_path_from_oxygen(droid_at_oxygen):
    seen = set()
    to_try = deque([(droid_at_oxygen, [])])
    longest_path = []
    while to_try:
        cpu, path = to_try.popleft()
        if not path:
            result = Sense.Empty
        else:
            coords = get_coords(path)
            if coords in seen:
                continue
            seen.add(coords)
            move = path[-1]
            cpu.input.put(move)
            cpu.run(pause_on_output=True)
            result = Sense(cpu.output.get())
        if result == Sense.Wall:
            continue
        else:
            if len(path) > len(longest_path):
                longest_path = path
            for d in Move:
                to_try.append((cpu.fork(), path + [d]))
    return longest_path

longest_path = longest_path_from_oxygen(repair_droid.fork())
''.join([step.name for step in longest_path])

'SSEEEESSWWWWWWNNNNNNNNWWSSSSSSSSWWNNNNNNNNNNNNNNEENNEENNEESSSSWWSSWWSSEEEEEESSWWSSEESSEENNNNNNNNWWNNEENNWWNNEENNWWNNEENNNNNNWWNNWWWWSSSSEESSWWWWNNNNWWNNEENNNNWWWWNNWWNNWWWWWWWWWWSSSSEESSEEEEEESSWWSSEESSSSWWWWNNWWSSWWWWNNWWNNWWWWNNEEEENNEENNWWNNNNWWWWWWSSEEEESSSSWWWWNNWWSSSSSSSSEESSEEEEEESSWWWWSSSSSSWWWWNNWWSSSSEESSWWSSEESSEEEESSEESSWWWWNNWWSSSSSSEEEENNEESSEE'

In [33]:
get_coords(longest_path)

(4, -24)

#### Part 2 Answer
Use the repair droid to get a complete map of the area.  
**How many minutes will it take to fill with oxygen?**

In [34]:
len(longest_path)

360